In [18]:
library(tidytext)
library(stringr)
library(tidyverse)
library(tibble)
library(ggplot2)
library(lubridate)
library(SnowballC)
library(RSQLite)
library(sparklyr)

In [19]:
sc <- spark_connect(master = "local")

In [20]:
system('head -n 1 ../../data/bitcoinoct17tooct18/all_sorted.csv', intern = TRUE)

[1] "Tweet ID, Conversation ID, Author Id , Author Name, isVerified, DateTime, Language, Tweet Text, Replies, Retweets, Favorites, Mentions, Hashtags, Permalink, URLs, isPartOfConversation, isReply, isRetweet, Reply To User ID, Reply To User Name, Quoted Tweet ID, Quoted Tweet User Name, Quoted Tweet User ID"

In [21]:
system('tail -n 1 ../../data/bitcoinoct17tooct18/all_sorted.csv', intern = TRUE)

[1] "1049449307043024896,1049449307043024896,768679436,tapatrade,0,2018-10-08 19:59,und,#bitcoin https://twitter.com/MustStopMurad/status/1022169639386836992 …,0,0,0,,#bitcoin,/tapatrade/status/1049449307043024896,https://twitter.com/MustStopMurad/status/1022169639386836992,0,0,1,,,1022169639386836992,MustStopMurad,844304603336232960"

In [22]:
#con = dbConnect(SQLite(), dbname="../../data/twitter.db")
tweets_df = dbGetQuery(con, 'select * from bitcoin where DateTime > "2018-03-03" and DateTime < "2018-05-03" and Language="en"')
#tweets_df <- spark_read_csv(sc, name = "test", '/home/apcloud44/latent_aspect_modeling/data/bitcoinoct17tooct18/all_sorted.csv')
head(tweets_df)

# Source:   lazy query [?? x 23]
# Database: spark_connection
  Tweet_ID Conversation_ID Author_Id Author_Name isVerified DateTime Language
     <dbl>           <dbl>     <dbl> <chr>            <int> <chr>    <chr>   
1  9.62e17         9.62e17   9.04e17 CryptoCoin…          0 2018-02… bg      
2  9.62e17         9.62e17   9.04e17 CryptoCoin…          0 2018-02… bg      
3  9.62e17         9.62e17   8.82e17 NowBitcoin…          0 2018-02… ca      
4  9.62e17         9.62e17   8.94e17 falileevam4          0 2018-02… de      
5  9.62e17         9.62e17   9.55e17 cryptoaudio          0 2018-02… en      
6  9.62e17         9.62e17   9.59e17 ADATicker            0 2018-02… en      
# ... with 16 more variables: Tweet_Text <chr>, Replies <int>, Retweets <int>,
#   Favorites <int>, Mentions <chr>, Hashtags <chr>, Permalink <chr>,
#   URLs <chr>, isPartOfConversation <chr>, isReply <chr>, isRetweet <chr>,
#   Reply_To_User_ID <chr>, Reply_To_User_Name <chr>, Quoted_Tweet_ID <chr>,
#   Quoted_T

In [23]:
#tweets_df <- read.csv('../../data/bitcoinoct17tooct18/all_sorted.csv', nrow=261079)
#tweets_df <- spark_read_csv(sc = sc, name='bitcoint_df', 
#                            path = spark_normalize_path('../../data/bitcoinoct17tooct18/all_sorted.csv'), 
#                                                        header = TRUE, delimiter = ",") 

#%>% filter(DateTime > "2018-03-03" & DateTime < "2018-05-03" & Language == "en")
#head(tweets_df)

In [24]:
remove_reg <- "&amp;|&lt;|&gt;"
unnest_reg  <- "([^A-Za-z_\\d#@']|'(?![A-Za-z_\\d#@]))"

tidy_tweets <- tweets_df %>% 


#collect %>%

filter(!str_detect(Tweet_Text, "^RT")) %>%
mutate(text = stringr::str_remove_all(Tweet_Text, remove_reg)) %>%



unnest_tokens(word, text, token = "tweets") %>%
mutate(word=wordStem(word, language = 'english')) %>%

filter(!word %in% stop_words$word,
     !word %in% str_remove_all(stop_words$word, "'"),
     str_detect(word, "[a-z]"))

ERROR: Error: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8.0 failed 1 times, most recent failure: Lost task 1.0 in stage 8.0 (TID 39, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:248)
	at org.apache.spark.scheduler.DirectTaskResult$$anonfun$writeExternal$1.apply$mcV$sp(TaskResult.scala:50)
	at org.apache.spark.scheduler.DirectTaskResult$$anonfun$writeExtern

In [ ]:
tidy_tweets$DateTime <- tidy_tweets$DateTime %>% as.Date(format = '%Y-%m-%d')
str(tidy_tweets)

In [ ]:
summary(tidy_tweets)

In [ ]:
tidy_tweets %>% count(DateTime) %>% 
ggplot() + aes(x=DateTime, y=n) + geom_line() +geom_point() + ylab('No of Tweets')


Calculate frequency of words against each month

In [ ]:
tidy_tweets_orig <- tidy_tweets %>% mutate(week = week(DateTime)) %>% group_by(week) %>% count(word, sort=TRUE) %>% 
left_join(tidy_tweets %>% mutate(week = week(DateTime)) %>% group_by(week) %>% summarize(total=n())) %>% 
mutate(freq = n/total) %>% mutate(logn = round(log(n))) %>% select(week, word, freq, n, logn, total)

In [ ]:
n=500
tidy_tweets_freq <- tidy_tweets_orig %>% arrange(desc(n))  %>% group_by(week) %>% top_n(freq, n=n) 
tidy_tweets_freq %>% arrange(week)

Let's convert words to columns (dimensions)

In [ ]:
tidy_tweets_plot <- tidy_tweets_freq %>% select(-n,-freq, -logn) %>% spread(week, freq, fill = 0)
tidy_tweets_ca <- tidy_tweets_freq %>% select(-n, -freq,-total) %>% spread(week, logn, fill = 0) 

In [ ]:
saveRDS(tidy_tweets_ca, file = paste0('../../data/tidy_tweets_per_week_log_bitcoin_',n,'.rds'))
tidy_tweets_ca

In [ ]:
saveRDS(tidy_tweets_plot, file = paste0('../../data/tidy_tweets_per_week_n_bitcoin_',n,'.rds'))
tidy_tweets_plot

In [ ]:
#tidy_tweets_plot %>% ggplot() + aes('9','10') + geom_jitter() + geom_text(aes(label = word), check_overlap = TRUE, vjust = 0)